# 3. Neural Networks with PyTorch

### 1. Prepare data

In [16]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.model_selection import train_test_split

df = pd.read_parquet('..\\Data\\data_01_clean.parquet')

X=df.drop(columns=['Dev'])
y= df['Dev']

X = X.values # to numpy-array
y = y.values # to numpy-array

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

###  2. Model Training

In [3]:
import torch
torch.manual_seed(0)

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [13]:
(X_test.shape,y_test.shape)

((54000, 39), (54000,))

In [25]:
list(zip(X_test,y_test))

[(array([1.39768768e+02, 1.00000000e+02, 6.94444180e-02, 1.65104175e+00,
         6.38213501e+01, 6.38258133e+01, 2.01672253e+03, 2.00000000e+03,
         3.41178055e+01, 1.20949078e+00, 0.00000000e+00, 0.00000000e+00,
         4.39133415e+01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 4.39133415e+01,
         2.15277771e+02, 2.23738602e+02, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 1.96701384e+00, 2.00000000e+00,
         0.00000000e+00, 1.98849821e+00, 2.00000000e+00, 2.23738602e+02,
         8.25636578e+00, 1.50000000e+01, 2.75141277e+01, 1.91416025e-01,
         2.00000003e-01, 0.00000000e+00, 1.73979043e-03]),
  1),
 (array([1.18754349e+02, 1.00000000e+02, 6.65509701e-02, 2.81944442e+00,
         3.11474724e+01, 3.11395512e+01, 2.02640417e+03, 2.00000000e+03,
         2.54559727e+01, 1.11689806e+00, 7.16177292e+01, 0.00000000e+00,
         5.66535835e+01, 5.87022583e+02, 7.16177292e+01, 6.

In [7]:
# train_x = torch.Tensor(X_train).float().to(device)
# test_x = torch.Tensor(X_test).float().to(device)
# train_y = torch.Tensor(y_train).long().to(device)
# test_y = torch.Tensor(y_test).long().to(device)

# THIS WORKS FINE

In [28]:
batch_size = 4
#trainloader = torch.utils.data.DataLoader(, batch_size=batch_size,
 #                                         shuffle=True, num_workers=2)
                                          
testloader = torch.utils.data.DataLoader(list(zip(X_test,y_test)), batch_size=batch_size,
                                          shuffle=True, num_workers=2)

In [8]:
import torch.nn as nn
import torch.nn.functional as F


class DeepNeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(39, 4096)
        self.hidden2 = nn.Linear(4096, 1000)
        self.hidden3 = nn.Linear(1000, 100)
        self.hidden4 = nn.Linear(100, 2)

    def forward(self, x):
        
        z1 = self.hidden1(x)
        a1 = torch.sigmoid(z1)
        z2 = self.hidden2(a1)
        a2 = torch.sigmoid(z2)
        z3 = self.hidden3(a2)
        a3 = torch.sigmoid(z3)
        z4 = self.hidden4(a3)
        a4 = torch.sigmoid(z4)
        z5 = self.output(a4)
        return z5

In [9]:
import time

dnn = DeepNeuralNetwork()
dnn.to(device)            # copy the model to the device 
dnn.train()      # set model into training mode

no_epochs = 20
learning_rate = 0.001

loss_func = nn.CrossEntropyLoss()  

optimizer = torch.optim.SGD(dnn.parameters(), lr=learning_rate)

start_time = time.time()
losses = []
for iteration in range(no_epochs):
    
    optimizer.zero_grad()
    y_hat = dnn(train_x) # we predict on all data points (= batch gradient descent)
    
    loss = loss_func(y_hat, train_y) # calculate the loss
    loss.backward() # backpropagate the loss to calculate gradients
    optimizer.step() # update the weights using these gradients 

    losses.append(loss.item())
    if iteration % 20 == 0:
        print(f"Loss in epoch {iteration} is {loss.item()}")

KeyboardInterrupt: 

After the network is trained, we can use it to predict on the test data.

In [ ]:
dnn.eval() # set network to evaluation mode
y_pred = dnn(test_x)
predicted = torch.argmax(y_pred.data, 1)
correct = (predicted == test_y).sum().item()
print(f"Accuarcy is {100. * correct / len(test_x)}%")

In [ ]:
PATH = './ApuP_net.pth'
torch.save(dnn.state_dict(), PATH)